## Covert to FEN

In [1]:
!pip install chess

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 41.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for chess: filename=chess-1.11.2-py3-none-any.whl size=147775 sha256=0cff6bd67c0940c61cb230e4b390c8d116b2d3eaa32a08d89bf0e5d379bb3673
  Stored in directory: /root/.cache/pip/wheels/fb/5d/5c/59a62d8a695285e59ec9c1f66add6f8a9ac4152499a2be0113
Successfully built chess


In [2]:
import bz2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import math
import os, io, re, json, glob
from typing import Iterator, Optional, List, Dict, Any
import chess, chess.pgn

In [3]:
with bz2.open("/kaggle/input/raw-chess-games-pgn/lichess_db_standard_rated_2014-08.pgn.bz2", "rb") as f:
    data = f.read()

data = str(data) # Convert binary data into string for easier functionality
raw_games = data.split('[Event') # Split the data into chess games using the '[Event' string
print("Game at 0th index: %s" % raw_games[0])
del raw_games[0] # The first index isn't a game
del data # Remove binary string to save memory

eval_games = []
for game in raw_games:
    if game.find('eval') != -1:
        eval_games.append(game)

Game at 0th index: b'


In [4]:
print(eval_games[0])

 "Rated Bullet game"]\n[Site "https://lichess.org/s3CHmrgH"]\n[White "JekyllHyde"]\n[Black "Maconouchi"]\n[Result "1-0"]\n[UTCDate "2014.07.31"]\n[UTCTime "22:01:10"]\n[WhiteElo "1627"]\n[BlackElo "1662"]\n[WhiteRatingDiff "+52"]\n[BlackRatingDiff "-13"]\n[ECO "B01"]\n[Opening "Scandinavian Defense: Modern Variation #2"]\n[TimeControl "60+0"]\n[Termination "Normal"]\n\n1. e4 { [%eval 0.2] } 1... d5 { [%eval 0.47] } 2. exd5 { [%eval 0.45] } 2... Nf6 { [%eval 0.58] } 3. Nc3 { [%eval 0.38] } 3... Nxd5 { [%eval 0.32] } 4. Nf3 { [%eval 0.27] } 4... Nc6 { [%eval 0.52] } 5. d4 { [%eval 0.43] } 5... e6 { [%eval 0.51] } 6. Be3 { [%eval 0.13] } 6... Bb4 { [%eval 0.26] } 7. Bd2 { [%eval 0.22] } 7... O-O { [%eval 0.29] } 8. a3 { [%eval -0.04] } 8... Bxc3 { [%eval 0.0] } 9. Bxc3 { [%eval -0.49] } 9... Nxc3 { [%eval -0.42] } 10. bxc3 { [%eval -0.43] } 10... Qf6 { [%eval 0.03] } 11. Bd3 { [%eval -0.27] } 11... h6?! { [%eval 0.57] } 12. O-O { [%eval 0.41] } 12... Re8 { [%eval 0.65] } 13. Qe2 { [%eval 

In [6]:
EVAL_RE = re.compile(r"\[%eval\s+([+#\-0-9\.]+)\]")

def extract_moves_from_pgn(path_in: str, path_out: str, limit=None):
    n_games, n_rows = 0, 0
    with bz2.open(path_in, "rb") as f:
        fh = io.TextIOWrapper(f, encoding="utf-8", errors="ignore")
        with open(path_out, "w", encoding="utf-8") as fout:
            while True:
                game = chess.pgn.read_game(fh)
                if game is None:
                    break
                n_games += 1
                board = game.board()
                node = game
                ply = 0
                while node.variations:
                    next_node = node.variation(0)
                    move = next_node.move
                    fen_before = board.fen()
                    move_uci = move.uci()
                    fout.write(json.dumps({
                        "fen": fen_before,
                        "move": move_uci,
                        "side_to_move": 1 if board.turn else -1
                    }) + "\n")
                    board.push(move)
                    node = next_node
                    ply += 1
                    n_rows += 1
                if limit and n_games >= limit:
                    break
    print(f"[OK] {n_games} games, {n_rows} moves → {path_out}")

# Demo với 1 file lichess .pgn.bz2
extract_moves_from_pgn(
    "/kaggle/input/raw-chess-games-pgn/lichess_db_standard_rated_2014-08.pgn.bz2",
    "/kaggle/working/move_dataset.jsonl",
    limit=100000
)


[OK] 100000 games, 6731478 moves → /kaggle/working/move_dataset.jsonl


_____________________________________________________________________________________________________

## Setup Data

In [2]:
!pip install python-chess tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 51.9 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for chess: filename=chess-1.11.2-py3-none-any.whl size=147775 sha256=d33db72334501ad9eb01278e763239495eddd7bbd737236006378f61765d4a35
  Stored in directory: /root/.cache/pip/wheels/fb/5d/5c/59a62d8a695285e59ec9c1f66add6f8a9ac4152499a2be0113
Successfully built chess


In [3]:
import os, json

data_path = "/kaggle/input/move-dt/move_dataset.jsonl"
out_dir = "/kaggle/working/chess_prep"
os.makedirs(out_dir, exist_ok=True)

seen = set()
kept = 0
removed = 0

out_file = os.path.join(out_dir, "move_dataset_clean.jsonl")
with open(data_path, "r") as fin, open(out_file, "w") as fout:
    for line in fin:
        obj = json.loads(line)
        key = (obj["fen"], obj["move"], obj["side_to_move"])
        if key in seen:
            removed += 1
            continue
        seen.add(key)
        kept += 1
        fout.write(json.dumps(obj) + "\n")

print(f"Saved cleaned dataset: {out_file}")
print(f"Kept: {kept}, Removed: {removed}, Total: {kept+removed}")


Saved cleaned dataset: /kaggle/working/chess_prep/move_dataset_clean.jsonl
Kept: 5968500, Removed: 762978, Total: 6731478


In [4]:
import os, json, random
from typing import List, Dict, Any

def split_jsonl_dataset(
    in_file: str,
    out_dir: str,
    seed: int = 42,
    train_ratio: float = 0.8,
    val_ratio: float = 0.1,
) -> None:

    with open(in_file, "r", encoding="utf-8") as f:
        data: List[Dict[str, Any]] = [json.loads(line) for line in f]

    random.seed(seed)
    random.shuffle(data)

    n = len(data)
    n_train = int(train_ratio * n)
    n_val   = int(val_ratio * n)

    splits = {
        "train.jsonl": data[:n_train],
        "val.jsonl":   data[n_train:n_train + n_val],
        "test.jsonl":  data[n_train + n_val:],
    }

    os.makedirs(out_dir, exist_ok=True)
    for name, subset in splits.items():
        path = os.path.join(out_dir, name)
        with open(path, "w", encoding="utf-8") as f:
            for obj in subset:
                f.write(json.dumps(obj, ensure_ascii=False) + "\n")
        print(f"Saved {name}: {len(subset)} samples")


split_jsonl_dataset("/kaggle/working/chess_prep/move_dataset_clean.jsonl", "/kaggle/working/chess_prep")


Saved train.jsonl: 4774800 samples
Saved val.jsonl: 596850 samples
Saved test.jsonl: 596850 samples


In [8]:
import chess, json, os

def build_alphazero_4672():
    move2id = {}
    id2move = {}
    idx = 0
    
    directions = [
        (1,0),(0,1),(-1,0),(0,-1),   # rook
        (1,1),(-1,1),(1,-1),(-1,-1)  # bishop
    ]
    knight_offsets = [(2,1),(1,2),(-1,2),(-2,1),(-2,-1),(-1,-2),(1,-2),(2,-1)]
    promo_pieces = [chess.ROOK, chess.BISHOP, chess.KNIGHT]  # queen promotion coi như mặc định
    
    for sq in chess.SQUARES:
        r0, c0 = divmod(sq, 8)
        
        # sliding moves (56 = 8 directions × 7 steps)
        for dr,dc in directions:
            for k in range(1,8):
                r, c = r0+dr*k, c0+dc*k
                if 0 <= r < 8 and 0 <= c < 8:
                    to_sq = r*8+c
                    uci = chess.Move(sq, to_sq).uci()
                else:
                    uci = f"null_{sq}_{dr}_{dc}_{k}"  # dummy move
                move2id[uci] = idx
                id2move[idx] = uci
                idx += 1

        # knight moves (8)
        for dr,dc in knight_offsets:
            r, c = r0+dr, c0+dc
            if 0 <= r < 8 and 0 <= c < 8:
                to_sq = r*8+c
                uci = chess.Move(sq, to_sq).uci()
            else:
                uci = f"null_knight_{sq}_{dr}_{dc}"
            move2id[uci] = idx
            id2move[idx] = uci
            idx += 1

        # underpromotions (9 = 3 dirs × 3 promos)
        for dc in [-1,0,1]:
            for promo in promo_pieces:
                if r0 == 6:  # white pawn promotion
                    r, c = r0+1, c0+dc
                    if 0 <= c < 8:
                        to_sq = r*8+c
                        uci = chess.Move(sq, to_sq, promotion=promo).uci()
                    else:
                        uci = f"null_promo_w_{sq}_{dc}_{promo}"
                elif r0 == 1:  # black pawn promotion
                    r, c = r0-1, c0+dc
                    if 0 <= c < 8:
                        to_sq = r*8+c
                        uci = chess.Move(sq, to_sq, promotion=promo).uci()
                    else:
                        uci = f"null_promo_b_{sq}_{dc}_{promo}"
                else:
                    uci = f"null_promo_{sq}_{dc}_{promo}"
                move2id[uci] = idx
                id2move[idx] = uci
                idx += 1

    print(f"Vocab size: {len(move2id)}")  # phải ra 4672
    return move2id, id2move

move2id, id2move = build_alphazero_4672()

WORKDIR = "/kaggle/working/chess_prep"
with open(os.path.join(WORKDIR, "move_vocab_4672.json"), "w") as f:
    json.dump(move2id, f)


Vocab size: 4672


In [17]:
def unknown_rate(path, move2id):
    total = 0
    unk = 0
    with open(path, "r", encoding="utf-8") as f:
        for line in f:
            total += 1
            m = json.loads(line)["move"]
            if m not in move2id:
                unk += 1
    rate = (unk / total) if total else 0.0
    return total, unk, rate

val_tot, val_unk, val_rate = unknown_rate(VAL, move2id)
test_tot, test_unk, test_rate = unknown_rate(TEST, move2id)

stats = {
    "dedup_kept": 5968500,        # điền giá trị bạn đã in được, hoặc đọc lại từ file nếu muốn
    "dedup_removed": 762978,
    "vocab_size": len(move2id),
    "val_total": val_tot, "val_unknown": val_unk, "val_unknown_rate": val_rate,
    "test_total": test_tot, "test_unknown": test_unk, "test_unknown_rate": test_rate,
}

with open(STATS, "w", encoding="utf-8") as f:
    json.dump(stats, f, indent=2, ensure_ascii=False)

print(json.dumps(stats, indent=2))


{
  "dedup_kept": 5968500,
  "dedup_removed": 762978,
  "vocab_size": 4672,
  "val_total": 596850,
  "val_unknown": 1551,
  "val_unknown_rate": 0.002598642875094245,
  "test_total": 596850,
  "test_unknown": 1555,
  "test_unknown_rate": 0.00260534472648069
}


In [12]:
import os, json, numpy as np, torch, chess
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F

class FenMoveDataset(Dataset):
    def __init__(self, path: str, move2id: dict):
        with open(path, "r", encoding="utf-8") as f:
            self.samples = [json.loads(line) for line in f]
        self.move2id = move2id

    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        obj = self.samples[idx]
        x, board = self.fen_to_planes(obj["fen"])
        y = self.move2id.get(obj["move"], -1)
        mask = self.legal_mask(board)
        return x.astype(np.float32), y, mask

    @staticmethod
    def fen_to_planes(fen: str):
        board = chess.Board(fen)
        planes = []
        piece_types = [chess.PAWN,chess.KNIGHT,chess.BISHOP,
                       chess.ROOK,chess.QUEEN,chess.KING]
        for color in [chess.WHITE, chess.BLACK]:
            for pt in piece_types:
                m = np.zeros((8,8), dtype=np.float32)
                for sq in board.pieces(pt, color):
                    r,c = divmod(63 - sq, 8)
                    m[r,c] = 1.0
                planes.append(m)
        # side to move
        planes.append(np.full((8,8), 1.0 if board.turn==chess.WHITE else 0.0, dtype=np.float32))
        # castling rights
        wk = np.full((8,8), board.has_kingside_castling_rights(chess.WHITE), dtype=np.float32)
        wq = np.full((8,8), board.has_queenside_castling_rights(chess.WHITE), dtype=np.float32)
        bk = np.full((8,8), board.has_kingside_castling_rights(chess.BLACK), dtype=np.float32)
        bq = np.full((8,8), board.has_queenside_castling_rights(chess.BLACK), dtype=np.float32)
        planes += [wk,wq,bk,bq]
        # en passant
        ep = np.zeros((8,8), dtype=np.float32)
        if board.ep_square is not None:
            r,c = divmod(63 - board.ep_square, 8)
            ep[r,c] = 1.0
        planes.append(ep)
        return np.stack(planes, axis=0), board

    def legal_mask(self, board: chess.Board):
        mask = np.zeros(len(self.move2id), dtype=np.float32)
        for m in board.legal_moves:
            uci = m.uci()
            if uci in self.move2id:
                mask[self.move2id[uci]] = 1.0
        return mask


def collate_fn(batch):
    xs, ys, masks = zip(*batch)
    return (
        torch.from_numpy(np.stack(xs)).float(),
        torch.tensor(ys, dtype=torch.long),
        torch.from_numpy(np.stack(masks)).float()
    )


In [13]:
class PolicyNet(nn.Module):
    def __init__(self, vocab_size: int):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(18, 128, 3, padding=1), nn.ReLU(),
            nn.Conv2d(128,128,3,padding=1), nn.ReLU(),
            nn.Conv2d(128,128,3,padding=1), nn.ReLU(),
            nn.Flatten()
        )
        self.fc = nn.Sequential(
            nn.Linear(128*8*8, 1024), nn.ReLU(),
            nn.Linear(1024, vocab_size)
        )

    def forward(self, x):
        return self.fc(self.conv(x))


In [14]:
class MaskedCrossEntropyLoss(nn.Module):
    def forward(self, logits, target, legal_mask):
        masked_logits = logits.masked_fill(legal_mask==0, float("-inf"))
        log_probs = F.log_softmax(masked_logits, dim=-1)
        B,V = logits.shape
        idx = torch.arange(B, device=logits.device)
        valid = (target >= 0) & (target < V)
        tgt_logp = log_probs[idx, target]
        tgt_logp[~valid] = 0.0
        if valid.any():
            return -(tgt_logp[valid].mean())
        else:
            return torch.tensor(0.0, device=logits.device)

# ============================================================
# 4. Trainer
# ============================================================

class Trainer:
    def __init__(self, model, train_dl, val_dl, device="cpu", lr=1e-3):
        self.model = model.to(device)
        self.train_dl = train_dl
        self.val_dl = val_dl
        self.device = device
        self.opt = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
        self.criterion = MaskedCrossEntropyLoss()

    def run_epoch(self, train=True):
        self.model.train(train)
        total_loss, total = 0.0, 0
        with torch.set_grad_enabled(train):
            for X,y,mask in self.train_dl if train else self.val_dl:
                X,y,mask = X.to(self.device), y.to(self.device), mask.to(self.device)
                logits = self.model(X)
                loss = self.criterion(logits, y, mask)
                if train:
                    self.opt.zero_grad()
                    loss.backward()
                    self.opt.step()
                bs = X.size(0)
                total_loss += loss.item() * bs
                total += bs
        return total_loss / max(total,1)

    def fit(self, epochs=5):
        for ep in range(epochs):
            tr = self.run_epoch(True)
            va = self.run_epoch(False)
            print(f"Epoch {ep}: train {tr:.4f} | val {va:.4f}")

In [15]:
import torch
import torch.nn as nn, torch.nn.functional as F, math
from torch.amp import autocast, GradScaler
from pathlib import Path

class ResBlock(nn.Module):
    def __init__(self, ch):
        super().__init__()
        self.f = nn.Sequential(
            nn.Conv2d(ch, ch, 3, padding=1, bias=False), nn.BatchNorm2d(ch), nn.ReLU(inplace=True),
            nn.Conv2d(ch, ch, 3, padding=1, bias=False), nn.BatchNorm2d(ch),
        )
        self.act = nn.ReLU(inplace=True)
    def forward(self, x): 
        return self.act(x + self.f(x))

class ChessResNet(nn.Module):
    def __init__(self, in_ch=18, ch=160, n=12, vocab=100000):
        super().__init__()
        self.stem = nn.Sequential(
            nn.Conv2d(in_ch, ch, 3, padding=1, bias=False),
            nn.BatchNorm2d(ch), nn.ReLU(inplace=True),
        )
        self.trunk = nn.Sequential(*[ResBlock(ch) for _ in range(n)])
        self.policy = nn.Sequential(
            nn.Conv2d(ch, 32, 1), nn.ReLU(),
            nn.Flatten(),
            nn.Linear(32*8*8, vocab)
        )
    def forward(self, x):
        h = self.trunk(self.stem(x))
        return self.policy(h)

def masked_ce(logits, targets, legal, label_smoothing=0.05):
    # CE chỉ tính trên legal moves + có label smoothing để bớt overfit
    masked = torch.full_like(logits, float("-inf"))
    for i, ids in enumerate(legal):
        if ids: masked[i, ids] = logits[i, ids]
        else:   masked[i] = logits[i]  # fallback
    return F.cross_entropy(masked, targets, label_smoothing=label_smoothing)

device = "cuda" if torch.cuda.is_available() else "cpu"

model = ChessResNet(in_ch=18, ch=160, n=12, vocab=len(move2id)).to(device)

opt = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)
scaler = GradScaler("cuda", enabled=(device=="cuda"))

@torch.no_grad()
def evaluate():
    model.eval()
    tot_loss=n=0; top1=top3=0
    for xb, yb, legal in val_dl:
        xb, yb = xb.to(device), yb.to(device)

        # sửa nhãn nếu không thuộc legal (hiếm)
        safe_y = yb.clone()
        for i,(yi,L) in enumerate(zip(yb.tolist(), legal)):
            if (yi<0) or (yi not in L):
                safe_y[i] = torch.tensor(L[0] if L else 0, device=device)

        with autocast("cuda", enabled=(device=="cuda")):
            logits = model(xb)
            loss = masked_ce(logits, safe_y, legal, label_smoothing=0.05)

        B = xb.size(0); tot_loss += float(loss)*B; n += B

        # Top-k trong legal
        for i in range(B):
            ids = legal[i] if legal[i] else torch.arange(logits.size(1)).tolist()
            li = logits[i, ids]
            k = min(3, len(ids))
            topk = torch.topk(li, k=k).indices.cpu().tolist()
            pred1 = ids[topk[0]]; gold = int(safe_y[i])
            if pred1==gold: top1+=1
            if gold in [ids[j] for j in topk]: top3+=1

    loss = tot_loss/max(1,n); ppl = math.exp(min(20,loss))
    return {"val_loss":loss, "val_ppl":ppl, "val_top1":top1/n, "val_top3":top3/n}


In [ ]:
# === MAIN TRAINING CELL ===

WORKDIR = "/kaggle/working/chess_prep"
TRAIN_PATH = os.path.join(WORKDIR, "train.jsonl")
VAL_PATH   = os.path.join(WORKDIR, "val.jsonl")
VOCAB_PATH = os.path.join(WORKDIR, "move_vocab_4672.json")

# Load vocab
with open(VOCAB_PATH, "r") as f:
    move2id = json.load(f)
vocab_size = len(move2id)
print(f"Loaded vocab size = {vocab_size}")

# Dataset + DataLoader
train_ds = FenMoveDataset(TRAIN_PATH, move2id)
val_ds   = FenMoveDataset(VAL_PATH, move2id)

train_dl = DataLoader(train_ds, batch_size=256, shuffle=True, num_workers=2, collate_fn=collate_fn)
val_dl   = DataLoader(val_ds, batch_size=256, shuffle=False, num_workers=2, collate_fn=collate_fn)

# Model + Trainer
device = "cuda" if torch.cuda.is_available() else "cpu"
model = PolicyNet(vocab_size)
trainer = Trainer(model, train_dl, val_dl, device=device, lr=1e-3)

# Train
trainer.fit(epochs=30)

# Save model checkpoint
CKPT_PATH = os.path.join(WORKDIR, "policy_model.pt")
torch.save({
    "model_state": model.state_dict(),
    "optimizer_state": trainer.opt.state_dict(),
    "vocab": move2id
}, CKPT_PATH)

print(f"Model saved to {CKPT_PATH}")


Loaded vocab size = 4672
Epoch 0: train 2.4057 | val 2.2220
Epoch 1: train 2.1585 | val 2.1600
Epoch 2: train 2.0934 | val 2.1380
Epoch 3: train 2.0508 | val 2.1252
Epoch 4: train 2.0179 | val 2.1284
Epoch 5: train 1.9902 | val 2.1310
Epoch 6: train 1.9658 | val 2.1385
Epoch 7: train 1.9430 | val 2.1536
Epoch 8: train 1.9230 | val 2.1639
Epoch 9: train 1.9050 | val 2.1719
Epoch 10: train 1.8879 | val 2.1918
Epoch 11: train 1.8722 | val 2.1997
Epoch 12: train 1.8578 | val 2.2131
Epoch 13: train 1.8442 | val 2.2415
Epoch 14: train 1.8319 | val 2.2555
Epoch 15: train 1.8209 | val 2.2911
Epoch 16: train 1.8105 | val 2.2806
Epoch 17: train 1.8005 | val 2.2900
Epoch 18: train 1.7916 | val 2.3202
Epoch 19: train 1.7839 | val 2.3145
Epoch 20: train 1.7763 | val 2.3174
Epoch 21: train 1.7699 | val 2.3459
Epoch 22: train 1.7631 | val 2.3700
Epoch 23: train 1.7580 | val 2.3918
Epoch 24: train 1.7524 | val 2.3872
Epoch 25: train 1.7471 | val 2.4099
Epoch 26: train 1.7427 | val 2.4247
Epoch 27: tra

In [ ]:
@torch.no_grad()
def predict_move(fen: str, topk=5):
    model.eval()
    x,_ = fen_to_planes(fen)
    x = torch.from_numpy(x).unsqueeze(0).float().to(device)
    with autocast("cuda", enabled=(device=="cuda")):
        logits = model(x)
    # lọc theo legal
    board = chess.Board(fen)
    ids = [move2id[m.uci()] for m in board.legal_moves if m.uci() in move2id]
    li = logits[0, ids]
    k = min(topk, len(ids))
    idx = torch.topk(li, k=k).indices.cpu().tolist()
    inv_vocab = {v:k for k,v in move2id.items()}
    return [inv_vocab[ids[i]] for i in idx]

# ví dụ
print(predict_move("rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1", topk=5))
